In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-127984")
exp = Experiment(workspace = ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127984
Azure region: southcentralus
Subscription id: 55e71b9d-a209-42c0-8818-ca9cc885909c
Resource group: aml-quickstarts-127984


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4

def cluster_validation():
    cluster = 'my-cluster'
    try:
        compute_cluster = ComputeTarget(workspace = ws, name = cluster)
        print('Found existing compute cluster!')
    except ComputeTargetException:
        config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2',max_nodes = 4)
        compute_cluster = ComputeTarget.create(ws, cluster, config)
    compute_cluster.wait_for_completion(show_output = True)
    return compute_cluster

In [3]:
compute_cluster = cluster_validation()


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [66]:
import pandas as pd
train_data = pd.read_csv('train.csv')


In [99]:
test_data = pd.read_csv('testdata.csv')
test_data.drop('Unnamed: 0',axis = 1,inplace = True)
test_data

,Pclass,Sex,Age,Fare,Embarked,FamilySizeCategory,IsAlone,Title_1,Title_2,Title_3,Title_4,Title_5
0,3,1,2,0,2,2,1,1,0,0,0,0
1,3,0,2,0,0,2,0,0,0,1,0,0
2,2,1,3,1,2,2,1,1,0,0,0,0
3,3,1,1,1,0,2,1,1,0,0,0,0
4,3,0,1,1,0,2,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,1,2,1,0,2,1,1,0,0,0,0
414,1,0,2,3,1,2,1,0,0,0,0,1
415,3,1,2,0,0,2,1,1,0,0,0,0
416,3,1,2,1,0,2,1,1,0,0,0,0


In [67]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.00,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.00,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,nan,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.00,0,0,111369,30.00,C148,C


In [91]:
x = pd.read_csv('xtraindelete.csv')
y = pd.read_csv('ytraindelete.csv')
y.drop('Unnamed: 0',axis = 1,inplace = True)
train_data = pd.concat([x,y],axis = 1,ignore_index = False)
train_data.drop('Unnamed: 0',axis = 1,inplace = True)

train_data


,Pclass,Sex,Age,Fare,Embarked,FamilySizeCategory,IsAlone,Title_1,Title_2,Title_3,Title_4,Title_5,Survived
0,3,1,1,0,0,2,0,1,0,0,0,0,0
1,1,0,2,3,1,2,0,0,0,1,0,0,1
2,3,0,1,1,0,2,1,0,1,0,0,0,1
3,1,0,2,3,0,2,0,0,0,1,0,0,1
4,3,1,2,1,0,2,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,1,1,1,0,2,1,0,0,0,0,1,0
887,1,0,1,2,0,2,1,0,1,0,0,0,1
888,3,0,2,2,0,2,0,0,1,0,0,0,0
889,1,1,1,2,1,2,1,1,0,0,0,0,1


In [92]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = 'classification',
    primary_metric = 'accuracy',
    training_data = train_data,
    label_column_name = 'Survived',
    n_cross_validations = 7,
    compute_target = 'local',
    iterations = 5)

In [93]:
experiment = Experiment(ws, "automl_exp")
run = experiment.submit(config = automl_config, show_output = True)


Running on local machine
Parent Run ID: AutoML_0b59f34f-fcdd-4014-9bd4-1a1e7c921ab7

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing f

In [94]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: automl_exp,
Id: AutoML_0b59f34f-fcdd-4014-9bd4-1a1e7c921ab7_3,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    max_features='sqrt',
                                                                                                    max_leaf_nodes=None,
                                        

In [100]:
y_predict = fitted_model.predict(test_data)
print(y_predict.shape)

(418,)


In [101]:
submit = pd.DataFrame(y_predict,columns = ['Survived'],index = [i + 892 for i in range(0,418)])
submit.index.name = 'PassengerId'
submit.to_csv('AzureAutoMLCleanedData.csv')
